In [1]:
import pandas as pd, numpy as np
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
import pickle
import sys
import os
import gc

# Load Train Data and Labels

In [2]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float16,
          "room_coor_x": np.float16,
          "room_coor_y": np.float16,
          "screen_coor_x": np.float16,
          "screen_coor_y": np.float16,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }
use_col = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page',
           'room_coor_x', 'room_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid', 'level_group']

In [3]:
targets = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( targets.shape )
targets.head()

(424116, 4)


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [4]:
feature_df = pd.read_csv('/kaggle/input/featur/feature_sort.csv')

# Feature Engineer

In [5]:
def delt_time_def(df):
    df.sort_values(by=['session_id', 'elapsed_time'], inplace=True)
    df['d_time'] = df['elapsed_time'].diff(1)
    df['d_time'].fillna(0, inplace=True)
    df['delt_time'] = df['d_time'].clip(0, 103000)
    df['delt_time_next'] = df['delt_time'].shift(-1)
    return df

In [6]:
def feature_engineer(train, kol_f):
    global kol_col, kol_col_max
    kol_col = 9
    kol_col_max = 11+kol_f*2
    col = [i for i in range(0,kol_col_max)]
    new_train = pd.DataFrame(index=train['session_id'].unique(), columns=col, dtype=np.float16)  
    new_train[10] = new_train.index # "session_id"    

    new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.3)
    new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.8)
    new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)
    new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.65)
    new_train[4] = train.groupby(['session_id'])['hover_duration'].agg('mean')
    new_train[5] = train.groupby(['session_id'])['hover_duration'].agg('std')    
    new_train[6] = new_train[10].apply(lambda x: int(str(x)[:2])).astype(np.uint8) # "year"
    new_train[7] = new_train[10].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8) # "month"
    new_train[8] = new_train[10].apply(lambda x: int(str(x)[4:6])).astype(np.uint8) # "day"
    new_train[9] = new_train[10].apply(lambda x: int(str(x)[6:8])).astype(np.uint8) + new_train[10].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)/60
    new_train[10] = 0
    new_train = new_train.fillna(-1)
    
    return new_train

In [7]:
def feature_next_t(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:       
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()
    return new_train

In [8]:
def feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:      
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

In [9]:
def experiment_feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    if row_f['kol_col'] == 1: 
        maska = train[row_f['col1']] == row_f['val1']
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = (train[col1] == val1) & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

In [10]:
def feature_quest_otvet(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    g1 = 0.7 
    g2 = 0.3 

    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f):         
        row_f = feature_q.loc[i]
        new_train = feature_next_t_otvet(row_f, new_train, train, i < gran1, i <  gran2, i) 
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

In [11]:
def feature_engineer_new(new_train, train, feature_q, kol_f):
    g1 = 0.7 
    g2 = 0.3 
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f): 
        row_f = feature_q.loc[i]       
        new_train = feature_next_t(row_f, new_train, train, i < gran1, i <  gran2, i)         
    return new_train

In [12]:
def feature_quest(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    new_train = feature_engineer_new(new_train, train, feature_q, kol_f)
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

In [13]:
# params = {'learning_rate': 0.18611630919419395, 'iterations': 399, 'depth': 8, 
#           'random_strength': 71.11420208846125, #'bagging_temperature': 54.595216183492724,
#           'l2_leaf_reg': 5.906726208791711e-05, 'min_data_in_leaf': 501,
#           'colsample_bylevel': 0.7088201337781473, 'bootstrap_type': 'No',
#          'random_seed': 1123}

In [14]:
# def create_model(old_train, quests, models, list_kol_f):
    
#     kol_quest = len(quests)
#     # ITERATE THRU QUESTIONS
#     for q in quests:
#         print('### quest ', q, end='')
#         new_train = feature_engineer(old_train, list_kol_f[q])
#         train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
#         print (' ---- ', 'train_q.shape = ', train_x.shape)
           
#         # TRAIN DATA
#         train_users = train_x.index.values
#         train_y = targets.loc[targets.q==q].set_index('session').loc[train_users]

#         # TRAIN MODEL 
        
# #         model = CatBoostClassifier(**params)

#         model = CatBoostClassifier(
#             n_estimators = 300,
#             learning_rate= 0.045,
#             depth = 6
#         )
        
#         model.fit(train_x.astype('float32'), train_y['correct'], verbose=False)

#         # SAVE MODEL, PREDICT VALID OOF
#         models[f'{q}'] = model
#     print('***')
    
#     return models

In [15]:
# models = {}
best_threshold = 0.63

In [16]:
list_kol_f = {
    1:140,3:110,
    4:120, 5:220, 6:130, 7:110, 8:110, 9:100, 10:140, 11:120,
    14: 160, 15:160, 16:130, 17:140,
    2:100,12:100,13:100,18:100
             }

In [17]:
os.chdir(r'/kaggle/working/')

In [18]:
# df0_4 = pd.read_csv('/kaggle/input/featur/train_0_4t.csv', dtype=dtypes) 
# kol_lvl = (df0_4 .groupby(['session_id'])['level'].agg('nunique') < 5)
# list_session = kol_lvl[kol_lvl].index
# df0_4  = df0_4 [~df0_4 ['session_id'].isin(list_session)]
# df0_4 = delt_time_def(df0_4)

# quests_0_4 = [1,2,3] 
# # list_kol_f = {1:140,3:110}

# models = create_model(df0_4, quests_0_4, models, list_kol_f)
# del df0_4

In [19]:
# df5_12 = pd.read_csv('/kaggle/input/featur/train_5_12t.csv', dtype=dtypes)
# kol_lvl = (df5_12.groupby(['session_id'])['level'].agg('nunique') < 8)
# list_session = kol_lvl[kol_lvl].index
# df5_12 = df5_12[~df5_12['session_id'].isin(list_session)]
# df5_12 = delt_time_def(df5_12)
# quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11,12,13] 

# # list_kol_f = {4:110, 5:220, 6:120, 7:110, 8:110, 9:100, 10:140, 11:120}

# models = create_model(df5_12, quests_5_12, models, list_kol_f)
# del df5_12

In [20]:
# df13_22 = pd.read_csv('/kaggle/input/featur/train_13_22t.csv', dtype=dtypes) 
# kol_lvl = (df13_22 .groupby(['session_id'])['level'].agg('nunique') < 10)
# list_session = kol_lvl[kol_lvl].index
# df13_22  = df13_22 [~df13_22 ['session_id'].isin(list_session)]
# df13_22 = delt_time_def(df13_22)

# quests_13_22 = [14, 15, 16, 17,18] 
# # list_kol_f = {14: 160, 15:160, 16:105, 17:140}

# models = create_model(df13_22, quests_13_22, models, list_kol_f)


In [21]:
# # Saving a Model
# import os
# dr = '/kaggle/working/models'

# if not os.path.exists(dr):
#     os.mkdir(r'/kaggle/working/models')
    
# os.chdir(r'/kaggle/working/models')
# # for q in quests_0_4 + quests_5_12 + quests_13_22:
# #     print(q)
# #     fadf
# #     models[q].save_model(f'cat_model_{q}.bin')

 
# import pickle
# for q in range(1,19):
    
#     filename = str('cat_model_'+f'{q}'+'.bin')

#     pickle.dump(models[f'{q}'], open(filename, 'wb'))
      
# #     models[q].save_model('cat_model_'+f'{q}'+'.bin')
    

In [22]:
import os
os.chdir(r'/kaggle/working/')

import jo_wilder_310
jo_wilder_310.make_env.func_dict['__called__'] = False
env = jo_wilder_310.make_env()

iter_test = env.iter_test()

quests_0_4 = [1,2,3]
quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11,12,13] 
quests_13_22 = [14, 15, 16, 17,18] 

list_q = {'0-4':quests_0_4, '5-12':quests_5_12, '13-22':quests_13_22}
for (test, sam_sub) in iter_test:
    test = test.sort_values(by = 'index')
    sam_sub['question'] = [int(label.split('_')[1][1:]) for label in sam_sub['session_id']]    
    grp = test.level_group.values[0]   
#     sam_sub['correct'] = 1
#     sam_sub.loc[sam_sub.question.isin([5, 8, 10, 13, 15]), 'correct'] = 0  
    old_train = delt_time_def(test[test.level_group == grp])
       
    for q in list_q[grp]:
        
#         start4 = time.time()
        new_train = feature_engineer(old_train, list_kol_f[q])
        new_train = feature_quest_otvet(new_train, old_train, q, list_kol_f[q])
#         new_train = feature_quest(new_train, old_train, q, kol_f)
        os.chdir(r'/kaggle/input/nm-std-perf-pred-cb-2a/models') 
    
        filename = str('cat_model_'+f'{q}'+'.bin') 

        clf = pickle.load(open(filename, 'rb'))  

        p = clf.predict_proba(new_train.astype('float32'))[:,1] 
        
#         clf = models[f'{q}']
#         p = clf.predict_proba(new_train.astype('float32'))[:,1]        
        
         
             
        
        mask = sam_sub.question == q 
        x = int(p[0]>best_threshold)
        sam_sub.loc[mask,'correct'] = x      
        
        
    sam_sub = sam_sub[['session_id', 'correct']]  
    os.chdir('/kaggle/working/')
    env.predict(sam_sub)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


TBB Warning: The number of workers is currently limited to 1. The request for 3 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



In [23]:
! head submission.csv

session_id,correct
20090109393214576_q1,1
20090109393214576_q2,1
20090109393214576_q3,1
20090109393214576_q4,1
20090109393214576_q5,0
20090109393214576_q6,1
20090109393214576_q7,1
20090109393214576_q8,0
20090109393214576_q9,1


In [24]:
df = pd.read_csv('submission.csv')
print( df.shape )
df.iloc[-54:]

(54, 2)


,session_id,correct
0,20090109393214576_q1,1
1,20090109393214576_q2,1
2,20090109393214576_q3,1
3,20090109393214576_q4,1
4,20090109393214576_q5,0
5,20090109393214576_q6,1
6,20090109393214576_q7,1
7,20090109393214576_q8,0
8,20090109393214576_q9,1
9,20090109393214576_q10,0
